Inducción de Reglas de Asociación usando 1R
===

* *90 min* | Ultima modificación: Junio 25, 2019

Las técnicas de clasificación son utilizadas para resolver problemas en los cuales se debe determinar a cual clase pertenece una instancia a partir de los valores de sus atributos; entre los casos prácticos de aplicación se encuentran: el diagnóstico de enfermedades (el paciente tiene o no la enfermedad), la detección de fraudes, los sistemas de reconocimiento (el objeto es o no es una persona), el riesgo crediticio (el solicitante pagará o no pagará la deuda). En este tutorial se presenta el algoritmo 1R, el cual permite construir un clasificador de referencia basado en reglas de asociación, y contra el cual se pueden contrastar los resultados de otros algoritmos más complejos.

## Descripción del problema

El problema abordado se desea determina a que clase pertenece una nueva observación ($A$, $B$ o $C$), con base en una muestra de observaciones recolectada previamente. 

Se tiene un conjunto ficticio de 15 ejemplos para los cuales se tienen tres características $x_1$, $x_2$ y $x_3$ y su respectiva clase. Se desea determinar a que clase pertenece un nuevo punto con coordenadas $x_1$, $x_2$ y $x_3$. Si las clases representan la presencia (o no) de una enfermedad, las variables $x_i$ prodian representan, por ejemplo, los resultados de los exámenes médicos de los pacientes.

In [1]:
%%writefile data.csv
x1,x2,x3,y
a,e,h,A
a,e,i,C
a,e,j,B
a,f,h,A
a,f,i,A
a,g,h,A
a,g,h,B
b,e,i,B
b,f,i,B
b,f,j,B
b,g,j,C
c,e,i,C
c,f,j,C
c,g,h,B
c,g,i,B
c,g,j,C
d,f,j,A
d,g,j,A

Overwriting data.csv


## Definición de probabilidad como frecuencia

**Espacio muestral:** se define como el conjunto de todos los posibles resultados de un experimento.

**Pregunta.---** En la tirada de dos dados, ¿cuál es el espacio muestral?

**Evento:** Es cualquier colección de posibles resultados de un experimento (subconjunto del espacio muestral).

En su forma más simple, el concepto de probabilidad puede interpretarse como la frecuencia con que ocurre un evento. Por ejempo, si en la tirada de dos dados se definen como un evento la cantidad de posibles resultados que dan una suma determinada, entonces:

![tirada-dados](assets/tirada-dados.jpg)


    Pr(𝑋= 2) = 1 / 36   Pr(𝑋= 6) = 5 / 36   Pr(𝑋=10) = 3 / 36
    Pr(𝑋= 3) = 2 / 36   Pr(𝑋= 7) = 6 / 36   Pr(𝑋=11) = 2 / 36
    Pr(𝑋= 4) = 3 / 36   Pr(𝑋= 8) = 5 / 36   Pr(𝑋=12) = 1 / 36
    Pr(𝑋= 5) = 4 / 36   Pr(𝑋= 9) = 4 / 36

**Pregunta.---** Los soldados mediavales apostaban con dos dados de la siguiente manera: si el resultado es par {2, 4, 6, 8, 10, 12} ganaba el soldado A; y si el resultado es impar {3, 5, 7, 9, 11} ganaba el soldado B. ¿Quién tiene mayor probabilidad de ganar?

## Propiedades y definiciones básicas sobre probabilidad


* Todas las probabilidades deben estar entre $0$ y $1$: 


$$0 \le \text{Pr}(x_i) \le 1$$


* Las probabilidades de eventos mutuamente exclusivos (no pueden ocurrir simultáneamente) y colectivamente exhaustivos (cubren todo el universo de casos posibles) deben sumar la unidad:

$$\sum_{i=1}^n \text{Pr}(x_i) = 1$$


* Probabilidad condicional $\text{Pr}(A \; | \; B)$: probabilidad de que ocurra un evento $A$ sabiendo que otro evento $B$ ya ocurrio.


* Independencia: Si los eventos $A$ y $B$ son independientes:

$$\text{Pr}(A \; |  \; B) = \text{Pr}(A)$$

## Reglas de asociación para clasificación

La metodología 1R se basa en la partición del espacio de entrada usando una y sólo una de las variables (atributos) del problema. Para el problema planteado, una regla basada en el atributo `x1` podría ser:

    if x1 in {a, d}:  y = A
    if x1 in {b}:     y = B
    if x1 in {c}:     y = C


El algoritmo funciona de la siguiente forma: se toma el primer atributo $x_1$ y se divide en grupos por cada  valor que puede tomar dicho atributo, es decir, por `a`, `b`, `c`, y `d`; para cada atributo se determina a que clase es más probable que pertenezca los ejemplos y se asigna dicha clase a dicho atributo. Es decir, para cada atributo se cuentan cuántos ejemplos hay de cada categoría y se asigna la clase por mayoría, esto es, si hay cuatro ejemplos para la categoría `a` de $x_1$ y tres de ellos pertencen a la clase `A` y el restante a `C` se dice que `if x1 == a: y = A`; esto equivale a decir que si `x1 == a` es más probable que el ejemplo pertenezca a la clase sea `A`. Así, el clasificador basado en este atributo podría ser escrito como un sistema de reglas:

         #   x1   x2   x3    y  
    --------------------------

    if x1 == a:  y = A
    
         1    a    g    h    A
         2    a    e    i    A
         3    a    f    h    A
        11    a    e    i    C
        
    if x1 == b:  y = B
    
         6    b    e    i    B 
         7    b    f    i    B
        12    b    g    j    C
         8    b    f    i    B

    if x1 == c:  y = C
    
         9    c    g    h    B
        10    c    g    h    B        
        13    c    f    j    C
        14    c    g    h    C
        15    c    e    i    C
    
    if x1 == d:  y = A

         4    d    f    j    A
         5    d    g    j    A

       
Al agrupar por $x_1$, este conjunto de reglas se reescribe como:

    if x1 in {a, d}:
        y = A
    elif x1 in {b}:
        y = B
    else:
        y = C
        
Para el clasificador anterior se puede calcular una métrica de error.

Luego se toma el segundo atributo $x_2$ y se procede de igual forma para construir otro clasificador. El proceso se repite hasta obtener un clasificador por cada atributo. Se escoge el clasificador con mayor precisión.

## Métricas de desempeño de clasificadores

Para evaluar el desempeño en problemas de clasificación dicotómicos (dos clases mutuamente excluyentes) se usa la matriz de confusión:


             | Pronostico
             |  P     N
    ---------|------------  
          P  |  TP    FN          
    Real     |
          N  |  FP    TN
    
    TP - Verdadero positivo (correcto)
    TN - Verdadero negativo (correcto)
    FN - Falso negativo (mal clasificado)
    FP - Falso positivo (mal clasificado)
    
Esta tabla permite analizar el comportamiento del modelo.

La medición de la precisión del modelo permite estimar el desempeño del modelo ante nuevos datos.

* Tasa de éxito (porcentaje de patrones clasificados correctamente):


$$\text{success rate} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}$$

* Tasa de error (porcentaje total de patrones clasificados incorrectamente):

$$\text{error rate} = \frac{\text{FP} + \text{FN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}} = 1 - \text{accuracy}$$

* Precisión o valor predictivo positivo: Proporción de casos positivos que fueron verdaderamente positivos.


$$\text{precision} = \frac{\text{TP}}{\text{TP}  + \text{FP}}$$

* Valor predictivo negativo: Proporción de casos negativos que fueron verdaderamente negativos.


$$\text{negative predictive value} = \frac{\text{TN}}{\text{TN}  + \text{FN}}$$

* Sensibilidad, tasa verdadera positiva, recall: mide la proporción de ejemplos positivos que fueron correctamente clasificados.


$$\text{sensitibity} = \frac{\text{TP}}{\text{TP}  + \text{FN}}$$

* Especifidad o tasa verdadera negativa: mide la proporción de ejemplos negativos correctamente clasificados.


$$\text{specifity} = \frac{\text{TN}}{\text{TN}  + \text{FP}}$$

**Actividad.---** Para el ejemplo planteado, calcule las métricas de error descritas.

**Actividad.---** Escriba un clasificador de una regla para el siguiente problema y cómpute la matriz de confusión para el mejor modelo encontrado.

![assets/tree-exercise.jpg](assets/tree-exercise.jpg)


## Solución en Python

### Preparación y carga de datos

In [2]:
##
## Preparación
##
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
%matplotlib inline
%load_ext rpy2.ipython

In [3]:
##
## Carga el archivo como un DataFrame
##
df = pd.read_csv(
    "data.csv",
    sep = ',',         # separador de campos
    thousands = None,  # separador de miles para números
    decimal = '.')     # separador de los decimales para números

df.head()

x1 x2 x3  y
0  a  e  h  A
1  a  e  i  C
2  a  e  j  B
3  a  f  h  A
4  a  f  i  A

### Generación de las reglas

In [4]:
def oneR_fit(df, var, target):
    # df: pandas DataFrame
    # var: nombre de la columna para construir el clasificador
    # target: columna con las clases
    
    ## crea una copia para no modificar el df original
    df0 = df.copy()
    
    ## Columna para contar la cantidad de ocurrencias del target
    df0['counter'] = 0
    
    ## conteo 
    df0 = df0.groupby([var, target], as_index=False).count()[[var, target, 'counter']]
    
    ## genera un vector con el maximo conteo por cada valor de var
    df0_max_counter = df0.groupby([var], as_index=False)['counter'].transform(max)
    
    ## vector de T/F indicando si la fila contiene el valor maximo del conteo
    df0_idx = df0['counter'] == df0_max_counter['counter']
    
    ## Seleccionar var y target para los maximos
    df0 = df0.loc[df0_idx, [var, target]]
    
    ## obtiene solo la primera fila si varias clases 
    ## tiene la misma frecuencia maxima
    df0 = df0.groupby(var, as_index=False).first()
    
    ## convierte var en el indice de la tabla
    df0 = df0.set_index(var)
    
    return df0

In [5]:
## reglas para x1
rules_x1 = oneR_fit(df, 'x1', 'y')
rules_x1

y
x1   
a   A
b   B
c   C
d   A

In [6]:
## reglas para x2
rules_x2 = oneR_fit(df, 'x2', 'y')
rules_x2

y
x2   
e   B
f   A
g   B

In [7]:
## reglas para x3
rules_x3 = oneR_fit(df, 'x3', 'y')
rules_x3

y
x3   
h   A
i   B
j   C

### Predicción

In [8]:
def oneR_predict(df, rules):
    ## genera una copia del DataFrame
    df0 = df.copy()
    
    ## convierte la variable usada para construir las reglas
    ## en el indice de las filas del dataframe 
    df0 = df0.set_index(rules.index.name, drop=False)
    
    ## hace un left join con base en los valores de los 
    ## indices de las filas del df y rules
    df0 = df0.join(rules, rsuffix='_predicted_' + rules.index.name)
    df0.index = range(len(df0))
    
    return df0

In [9]:
df0 = df.copy()
df0 = oneR_predict(df0, rules_x1)  ## Pronostico usando x1
df0 = oneR_predict(df0, rules_x2)  ## Pronostico usando x2
df0 = oneR_predict(df0, rules_x3)  ## Pronostico usando x3
df0

x1 x2 x3  y y_predicted_x1 y_predicted_x2 y_predicted_x3
0   a  e  h  A              A              B              A
1   a  f  h  A              A              A              A
2   a  g  h  A              A              B              A
3   a  g  h  B              A              B              A
4   c  g  h  B              C              B              A
5   a  e  i  C              A              B              B
6   b  e  i  B              B              B              B
7   c  e  i  C              C              B              B
8   a  f  i  A              A              A              B
9   b  f  i  B              B              A              B
10  c  g  i  B              C              B              B
11  a  e  j  B              A              B              C
12  b  f  j  B              B              A              C
13  c  f  j  C              C              A              C
14  d  f  j  A              A              A              C
15  b  g  j  C              B              B              C
16  c  g  j  C              C              B              C
17  d  g  j  A              A              B              C

### Precisión

In [10]:
## cantidad de aciertos
print('x1 : ', (df0.y == df0.y_predicted_x1).sum())
print('x2 : ', (df0.y == df0.y_predicted_x2).sum())
print('x3 : ', (df0.y == df0.y_predicted_x3).sum())

x1 :  12
x2 :  8
x3 :  9


### Matriz de confusión

In [11]:
from sklearn.metrics import confusion_matrix

confusion_matrix(df0.y, df0.y_predicted_x1)

array([[6, 0, 0],
       [2, 3, 2],
       [1, 1, 3]])

In [12]:
confusion_matrix(df0.y, df0.y_predicted_x2)

array([[3, 3, 0],
       [2, 5, 0],
       [1, 4, 0]])

In [13]:
confusion_matrix(df0.y, df0.y_predicted_x3)

array([[3, 1, 2],
       [2, 3, 2],
       [0, 2, 3]])

## Solución usando el lenguaje R

In [14]:
%%R -i df
##
## Instalación del paquete:
##
##       install.packages("OneR")
##

##
## Carga la librería.
##
library(OneR)

##
## Crea el clasificador. La librería reporta la precisión 
## del clasificador si se usa cada uno de los atributos 
## (variables x) y el sistema de reglas obtenido. 
## La notación y ~ . indica que la variable y del dataframe es función de las restantes
##
clf <- OneR(y ~ ., data = df, verbose = TRUE)

##
## la salida del modelo indica la tasa de éxito para los clasificadores de todas las clases
## Igualmente se imprimen las reglas de decisión para el mejor clasificador
##
clf  


    Attribute Accuracy
1 * x1        66.67%  
2   x3        50%     
3   x2        44.44%  
---
Chosen attribute due to accuracy
and ties method (if applicable): '*'


Call:
OneR.formula(formula = y ~ ., data = df, verbose = TRUE)

Rules:
If x1 = a then y = A
If x1 = b then y = B
If x1 = c then y = C
If x1 = d then y = A

Accuracy:
12 of 18 instances classified correctly (66.67%)



In [15]:
%%R
##
## La función summary reporta información detallada
## de los resultados del modelo, junto con la matriz 
## de confusión
##
summary(clf)


Call:
OneR.formula(formula = y ~ ., data = df, verbose = TRUE)

Rules:
If x1 = a then y = A
If x1 = b then y = B
If x1 = c then y = C
If x1 = d then y = A

Accuracy:
12 of 18 instances classified correctly (66.67%)

Contingency table:
     x1
y       a   b   c   d Sum
  A   * 4   0   0 * 2   6
  B     2 * 3   2   0   7
  C     1   1 * 3   0   5
  Sum   7   4   5   2  18
---
Maximum in each column: '*'

Pearson's Chi-squared test:
X-squared = 12.064, df = 6, p-value = 0.06056



**Matriz de confusión**

In [16]:
%%R
library(gmodels)
CrossTable(x = df$y, 
           y = predict(clf,df),
           prop.chisq=FALSE)


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  18 

 
             | predict(clf, df) 
        df$y |         A |         B |         C | Row Total | 
-------------|-----------|-----------|-----------|-----------|
           A |         6 |         0 |         0 |         6 | 
             |     1.000 |     0.000 |     0.000 |     0.333 | 
             |     0.667 |     0.000 |     0.000 |           | 
             |     0.333 |     0.000 |     0.000 |           | 
-------------|-----------|-----------|-----------|-----------|
           B |         2 |         3 |         2 |         7 | 
             |     0.286 |     0.429 |     0.286 |     0.389 | 
             |     0.222 |     0.750 |     0.400 |           | 
             |     0.111 |     0.167 |     0.111 |           | 
-------------|-----------|-----------